## 1. Install needed resources

In [1]:

import os
import re
import pandas as pd
import numpy as np
from bat import bro_multi_log_reader
import tarfile

## 2. Data preparation

### 2.1 Download data from S3 and unzip newfiles

In [3]:
os.getcwd()
os.chdir('/Users/Rachel/Desktop/AtomicIOT/Rachel/Sagemaker/Data/TestFor2018')
os.getcwd()

'/Users/Rachel/Desktop/AtomicIOT/Rachel/Sagemaker/Data/TestFor2018'

In [34]:
# Functions to unzip files
def DownloadFile(bucket, year, startmonth, endmonth):
    s3 = boto3.resource('s3')
    year = str(year)
    endmonth = endmonth + 1
    for i in range(startmonth, endmonth):
        if i <= 9:
            i = '0' + str(i)
        else:
            i = str(i)
        DownloadFileName = year + '-' + i + ".tgz"
        SaveFileName = year + '-' + i + ".tgz"
        s3.Bucket(bucket).download_file(DownloadFileName,SaveFileName)

In [4]:
def OpenZipFile (year, startmonth, endmonth):
    year = str(year)
    endmonth = endmonth + 1
    for i in range (startmonth, endmonth):
        if i <= 9:
            i = '0' + str(i)
        else:
            i = str(i)
        FileName = year + '-' + i + ".tgz"
        FolderName = year + '-' + i
        with tarfile.open(FileName, "r") as tar_ref:
            tar_ref.extractall(FolderName)

In [ ]:
OpenZipFile(2018,1,3)

### 2.2 Get look table, mac frame and reading function ready

#### 2.2.1 Get IPLook and DeviceLook ready

In [5]:
    # prepare refer tables - IPLook and DeviceLook
    IPrefer = pd.read_excel('/Users/Rachel/Desktop/AtomicIOT/Rachel/Sagemaker/Data/IOT.xlsx', header=None)
    IPrefer.columns = ['refer', 'IOT', 'Manufactor', 'Device','Other']
    IPrefer.refer = IPrefer.refer.apply(lambda x: '192.168.0.' + x)
    IPrefer.loc[:, 'refer'] = IPrefer.refer.str.split('-', expand=True)[0]
    IPrefer = IPrefer.apply(lambda x: x.str.strip())
    IPLook = IPrefer[['refer', 'IOT']].set_index('refer').to_dict()
    DeviceLook = IPrefer[['refer', 'Device']].set_index('refer').to_dict()

#### 2.2.2 Get Macone and MacExa ready

In [10]:
    # prepare Macone and MacExa
    Oct_Mac = pd.read_csv('/Users/Rachel/Desktop/AtomicIOT/Rachel/Sagemaker/Data/OCT_mac.csv')
    Aug_Mac = pd.read_excel('/Users/Rachel/Desktop/AtomicIOT/Rachel/Sagemaker/Data/Aug_DHCP.xlsx')
    Aug_Mac.loc[:, 'IOT'] = [IPLook['IOT'][x] for x in Aug_Mac.assigned_ip]
    Aug_Mac.loc[:, 'Device'] = [DeviceLook['Device'][x] for x in Aug_Mac.assigned_ip]
    DupIP = Aug_Mac.groupby('assigned_ip').mac.count()[Aug_Mac.groupby('assigned_ip').mac.count() > 1].index.values
    DupMac = Aug_Mac.groupby('mac').assigned_ip.count()[Aug_Mac.groupby('mac').assigned_ip.count() > 1].index.values
    Aug_Mac.loc[Aug_Mac.mac == DupMac[0], 'Device'] = 'Gaming'
    Aug_Mac.loc[41, 'Device'] = 'Mobile'
    Macone = Aug_Mac[Aug_Mac.assigned_ip.apply(lambda x: x not in DupIP)]
    Macdup = Aug_Mac[Aug_Mac.assigned_ip.apply(lambda x: x in DupIP)]
    Macone = Macone.append(Macdup[Macdup.mac.apply(lambda x: x in DupMac[0])], ignore_index=True)
    MacExa = Macdup[~Macdup.mac.apply(lambda x: x in DupMac[0])]

#### 2.2.3 Get MultiLogToDataFrame ready

In [14]:
# Create class to read log files into dataframes
class MultiLogToDataFrame(pd.DataFrame):
    """MultiLogToDataFrame: Converts multiple Bro logs to a Pandas DataFrame
        Args:
            log_fllename (string): The full path to the Bro log
            ts_index (bool): Set the index to the 'ts' field (default = True)
        Notes:
            This class is fairly simple right now but will probably have additional
            functionality for formal type specifications and performance enhancements
    """
    def __init__(self, log_filename, ts_index=False):
        """Initialize the LogToDataFrame class"""

        # Create a bro reader on a given log file
        reader = bro_multi_log_reader.BroMultiLogReader(log_filename)

        # Create a Pandas dataframe from reader
        super(MultiLogToDataFrame, self).__init__(reader.readrows())

        # Set the index
        if ts_index:
            self.set_index('ts', inplace=True)

### 2.3 Get four cleaning functions ready

In [15]:
# Functions to clean 4 types of log files
def CleanConn(conn):
    #only external data
    conn = conn[conn.local_resp.apply(lambda x: not x)]
    conn = conn[conn.proto == 'tcp']
    #select column
    conn = conn.drop(['tunnel_parents', 'uid'],1)
    #modify column
    if conn.duration.dtypes == 'object':
        conn['duration'] = conn.duration.replace('-', '00:00:00.00').astype('str')
        conn['duration'] = [datetime.datetime.strptime(x, "%H:%M:%S.%f") if re.match('\d+:\d+:\d+.\d+', x)
                            else datetime.datetime.strptime(x, "%d days, %H:%M:%S.%f") if 'days' in x
                            else datetime.datetime.strptime(x, "%d day, %H:%M:%S.%f") if 'day' in x
                            else datetime.datetime.strptime(x, "%H:%M:%S") for x in conn.duration]
        conn['duration'] = conn.duration.apply(lambda x :
                                               datetime.timedelta(hours=x.hour, minutes=x.minute, seconds=x.second))
    conn.service = conn.service.replace('-',np.NaN)
    return conn

def CleanDns(dns):
    dns.is_copy = False
    #select column
    dns = dns[['id.orig_h', 'id.orig_p', 'id.resp_h', 'ts','id.resp_p', 'query']]
    #modify column
    dns.loc[:, 'query'] = dns['query'].str.replace('\*[\\\\x00]{1,}', 'Unknown')
    dns.loc[:, 'query'] = dns['query'].str.split('.').apply(lambda x: x[-2] if len(x)>1 else x[0])
    return dns

def CleanHttp(http):
    http.is_copy = False
    #select column
    http = http[['id.orig_h', 'id.orig_p', 'id.resp_h','ts', 'id.resp_p', 'uri', 'user_agent', 'host']]
    return http

def CleanSsl(ssl):
    #select column
    ssl = ssl[['id.orig_h','id.orig_p','id.resp_h','id.resp_p','ts','subject','cert_chain_fuids']]
    return ssl

### 2.4 Get mapping functions ready

In [16]:
def Aug_Mapping(conn,Macone,MacExa,IPLook):
    conn2 = conn[[True if x in IPLook['IOT'].keys() else False for x in conn['id.orig_h']]]
    conn2.is_copy=False
    conn2['mac'] = [Macone.mac[y].values for y in [Macone.assigned_ip == x for x in conn2['id.orig_h']]]
    conn2['IOT'] = [Macone.IOT[y].values for y in [Macone.assigned_ip == x for x in conn2['id.orig_h']]]
    conn2['Device'] = [Macone.Device[y].values for y in [Macone.assigned_ip == x for x in conn2['id.orig_h']]]
    #
    for index, row in conn2.reset_index()[['ts','id.orig_h','mac','IOT','Device']].iterrows():
        for i , r in MacExa.iterrows():
            if row['ts'] >= r['min_ts'] and row['ts'] <= r['max_ts'] and row['id.orig_h'] == r['assigned_ip']:
                row['mac'] = r['mac']
                row['IOT'] = r['IOT']
                row['Device'] = r['Device']
            else:
                continue
    return conn2

def Aug_Other(df, Macone, MacExa, IPLook):
    df2 = df[[True if x in IPLook['IOT'].keys() else False for x in df['id.orig_h']]]
    df2.is_copy = False
    df2['mac'] = [Macone.mac[y].values for y in [Macone.assigned_ip == x for x in df2['id.orig_h']]]
    #
    for index, row in df2.reset_index()[['ts', 'id.orig_h', 'mac', 'IOT', 'Device']].iterrows():
        for i, r in MacExa.iterrows():
            if row['ts'] >= r['min_ts'] and row['ts'] <= r['max_ts'] and row['id.orig_h'] == r['assigned_ip']:
                row['mac'] = r['mac']
            else:
                continue
    return df2

# Functions to map data
def Oct_Mapping(conn,Oct_Mac):
    conn2 = conn[[True if x in Oct_Mac.assigned_ip.values else False for x in conn['id.orig_h']]]
    conn2['mac'] = [Oct_Mac.mac[y].values[0] for y in [Oct_Mac.assigned_ip == x for x in conn2['id.orig_h']]]
    conn2['IOT'] = [Oct_Mac.IOT[y].values[0] for y in [Oct_Mac.assigned_ip == x for x in conn2['id.orig_h']]]
    conn2['Device'] = [Oct_Mac.Device[y].values[0] for y in [Oct_Mac.assigned_ip == x for x in conn2['id.orig_h']]]
    return conn2

def Oct_Other(df, Oct_Mac):
    df2 = df[[True if x in Oct_Mac.assigned_ip.values else False for x in df['id.orig_h']]]
    df2['mac'] = [Oct_Mac.mac[y].values[0] for y in [Oct_Mac.assigned_ip == x for x in df['id.orig_h']]]

## 3. Main Function for Execution 

In [36]:
def Main (filepath, year, startmonth, endmonth):
    endmonth = endmonth + 1
    for i in range (startmonth, endmonth):
        Preprocessing (filepath, year, i)

In [41]:
def Preprocessing(filepath, year, month):
    
    # Check Leap Year using If Statement
    if (( year%400 == 0)or (( year%4 == 0 ) and ( year%100 != 0))):
        # this year is a leap year, Feb range (1,30)
        if month not in [1,3,5,7,8,10,12]: # means range (1,31)
            if month not in [4,6,9,11]: # means Feb
                m = range(1, 30)
            else: 
                m = range(1, 31)
        else:
            m = range(1,32)
    else:
        #this year is not a leap year, Feb range (1,29)
        if month not in [1,3,5,7,8,10,12]: # means range (1,31)
            if month not in [4,6,9,11]: # means Feb
                m = range(1, 29)
            else: 
                m = range(1, 31)
        else:
            m = range(1,32)
  
    # get file in each day of a month
    year = str(year)
    
    if month <= 9 :
            month = '0'+str(month)
    else:
            month = str(month)
    
    for i in m:
        if i <= 9 :
            i = '0'+str(i)
        else:
            i = str(i)
        conn = MultiLogToDataFrame(filepath+ year + '-' + month + '/'+
                                   year + '-' + month + '-' + i +'/conn.*.log.gz')
        conn = CleanConn(conn)
        conn2 = Oct_Mapping(conn,Oct_Mac)
            
        dns = MultiLogToDataFrame(filepath+ year + '-' + month + '/'+
                                   year + '-' + month + '-' + i +'/dns.*.log.gz')
        dns = CleanDns(dns)
        dns2 = Oct_Mapping(dns,Oct_Mac)
            
        http = MultiLogToDataFrame(filepath+ year + '-' + month + '/'+
                                   year + '-' + month + '-' + i +'/http.*.log.gz')
        http = CleanHttp(http)
        http2 = Oct_Mapping(http,Oct_Mac)
            
        ssl = MultiLogToDataFrame(filepath+ year + '-' + month + '/'+
                                   year + '-' + month + '-' + i +'/ssl.*.log.gz')
        ssl = CleanSsl(ssl)
        ssl2 = Oct_Mapping(ssl,Oct_Mac)

        conn2.to_csv('/Users/Rachel/Desktop/AtomicIOT/Rachel/Sagemaker/Data/TestFor2018/SampleClean/conn_'+ month+'_'+i+'.csv',index=False)
        dns2.to_csv('/Users/Rachel/Desktop/AtomicIOT/Rachel/Sagemaker/Data/TestFor2018/SampleClean/dns_' + month+'_'+i+'.csv',index=False)
        http2.to_csv('/Users/Rachel/Desktop/AtomicIOT/Rachel/Sagemaker/Data/TestFor2018/SampleClean/http_'+ month+'_'+i+'.csv',index=False)
        ssl2.to_csv('/Users/Rachel/Desktop/AtomicIOT/Rachel/Sagemaker/Data/TestFor2018/SampleClean/ssl_'+ month+'_'+i+'.csv',index=False)

In [31]:
filepath='/Users/Rachel/Desktop/AtomicIOT/Rachel/Sagemaker/Data/TestFor2018/'

In [42]:
Main(filepath, 2018, 1, 2)

Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmplt7ko1ef...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmplt7ko1ef...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmph80jcbyn...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmph80jcbyn...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpwzwe_pso...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpwzwe_pso...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmppdfwth5f...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmppdfwth5f...


/Users/Rachel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Rachel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Rachel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp1kmt6n_m...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp1kmt6n_m...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpnsuomqr8...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpnsuomqr8...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpikjo5yl2...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpikjo5yl2...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp57eflnk_...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp57eflnk_...


/Users/Rachel/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp5aqv455j...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp5aqv455j...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp0ow4f0e4...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp0ow4f0e4...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmptk15lh8q...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmptk15lh8q...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpx256enxi...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpx256enxi...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpggu3j5v5...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpggu3j5v5...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpcgb5z5d2...
Removed temporary file /var/folders/4

...

Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpfudtkk1m...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpfudtkk1m...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmps7v827wg...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmps7v827wg...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpoohmp72o...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpoohmp72o...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp6_ml1klq...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp6_ml1klq...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp6rpwo5cm...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp6rpwo5cm...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpbn8t_vdo...
Removed temporary file /var/folders/4

Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp4jwebs0l...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp4jwebs0l...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpgaqc9_83...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpgaqc9_83...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpivjecslj...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpivjecslj...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpkb60zcs5...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpkb60zcs5...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp9l0emmd2...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp9l0emmd2...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpyygo53bs...
Removed temporary file /var/folders/4

Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp6vik_0am...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp6vik_0am...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp4jr8fa75...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp4jr8fa75...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp277oozxn...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp277oozxn...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp27ut_sba...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp27ut_sba...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpd0yj87ng...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpd0yj87ng...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpmbazedmn...
Removed temporary file /var/folders/4

Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp731_mvsk...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp731_mvsk...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp0u2ragx4...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp0u2ragx4...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpqim17656...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpqim17656...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp3bkiqfoc...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp3bkiqfoc...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp_4ihgtaq...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmp_4ihgtaq...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpt93bklo7...
Removed temporary file /var/folders/4

Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpqg9x4udv...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpqg9x4udv...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpb7p8oen7...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpb7p8oen7...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpxikf0dp1...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpxikf0dp1...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpy753_ab3...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpy753_ab3...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpq6phbqup...
Removed temporary file /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpq6phbqup...
Successfully monitoring /var/folders/4k/ghwl61px4wv6lxj31l1q2tlc0000gn/T/tmpuquja1kz...
Removed temporary file /var/folders/4

In [ ]:
DownloadFile('sagemaker-model-ml', 2018, 1, 6)
OpenZipFile (2018, 1, 6)
filepath='/Users/Rachel/Desktop/AtomicIOT/Rachel/Sagemaker/Data/TestFor2018/'
Main(filepath, 2018, 1, 6)